# JSON Based Analytics

The purpose of this exercise is to provide an example of how you organize, perform, and communicate your work. The exercise itself is not meant to be difficult or particularly time consuming. We value our time, and we value your time. Feel free to explain what you would do if you had more time. Please include all of the source code for your analysis in your report, and please describe all of the steps. We should be able to reproduce your analysis with little work using the content of your report.

This exercise asks some questions about a collection of records that simulate personal information such as name, address, and phone number. The associated data file contains records in JSON Lines format with one JSON object per line. The records contain nested fields, and different records might contain different fields. The high level concept of this exercise is to picture this data set as a structured table. Because each JSON record is independent, some or all of the columns in this table can have missing values. Below, we refer to the names of the columns in this table as named fields.

Please present your solution as a single report that includes code, descriptive text, and results. We encourage you to include explanations of your approach to each problem. Examples of acceptable formats are PDF or HTML files generated from RMarkdown, Juypter Notebooks, or any another method you prefer.

In [137]:
import numpy as np
import pandas as pd

In [84]:
import json
import gzip

data=[]
#read in JSON lines into list of dictionary objects:
with gzip.open("ida_wrangling_exercise_data.2017-02-13.jsonl.gz", "rb") as file:
    for line in file.readlines():
        data.append(json.loads(line.decode("utf-8")))

In [77]:
type(data) #list of dictionaries
len(data) #15000

#first form of dictionary: 'name' field is nested with {'firstname', 'lastname', 'middlename'}, 'address' not nested
data[1]

##second form of dictionary: 'name' field is nested with {'firstname', 'lastname', 'middlename'}, 'address' also nested
data[2]

##third form of dictionary: 'name' field is not nested, 'address' is nested
data[100]

{'address': {'city': 'North Michaelhaven',
  'state': 'WV',
  'street': '71497 Sarah Cliff Suite 797',
  'zip': '28208'},
 'email': 'lawrencematthew@gomez.net',
 'id': '51554d878b7d4001bd60e5f16a23edc2',
 'name': 'Nicole West',
 'phone': '1-879-969-1341x697',
 'record_date': '2006-11-05T21:44:32',
 'ssn': 'xxx-xx-6261'}

## 1. Reshape List of Dictionaries

Start by making a list of all of the nested named fields that appear in any record. Concatenate nested field names using a period '.' to defind named fields for nested records. Present the list in alphabetical order. For example, if our data file contained the following

{"name": "Jane Doe", "address": {"personal": {"street": "123 Main St.", "city": "Springfield"}}}
{"name": "John Doe", "email": "johndoe@example.com"}
{"name": {"first": "Anne", "last": "Smyth"}, "phone": "123-245-7890"}
then the ordered list of fields would be

["address.personal.city", "address.personal.street", "email", "name", "name.first", "name.last", "phone"]
Note that a top-level named field such as "name" could contain either text or a nested JSON object. Thus "name" and "name.first" could both exists as separate fields in your list.

In [80]:
#flatten nested dictionaries by concatenating keys by '.'
#https://stackoverflow.com/questions/6027558/flatten-nested-python-dictionaries-compressing-keys

import collections

def flatten(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

#flatten each dictionaries in the list data and append back into a list of flatten dictionaries:
list_of_flat_dic=[]
for dic in data:
    list_of_flat_dic.append(flatten(dic))
    
list_of_flat_dic[0]

{'address.city': 'Hoodburgh',
 'address.state': 'RI',
 'address.street': '86314 David Pass Apt. 211',
 'address.zip': '83973',
 'dob': '1971-06-30',
 'email': 'opark@hotmail.com',
 'id': '01d68a4c598a45559c06f4df0b3d82cb',
 'name.firstname': 'Cynthia',
 'name.lastname': 'Dawson',
 'name.middlename': 'Claire',
 'phone': '624-869-4610',
 'record_date': '2006-07-08T09:02:13',
 'ssn': 'xxx-xx-2412'}

In [81]:
len(list_of_flat_dic) #Sanity check: 150000 dictionary items in list

150000

## 2. Answer the following questions for each field in your list from question 1.
["address.personal.city", "address.personal.street", "email", "name", "name.first", "name.last", "phone"]

- What percentage of the records contain the field?
- What are the five most common values of the field?

In the example data above, the field "address.personal.city" occurs in 1 out of 3 records so the fraction of records containing that field is 1/3. The exact percentage is 100/3. For this exercise, please approximate numeric answers to a reasonable precision such as 33.3%. Note that the field "name" occurs in the first and second record but not in the third record where the fields "name.first" and "name.last" are present. Thus, the "name" field occurs in 2/3 or roughly 66.6% of the records in the example above.

In [99]:
#extract values of field 'address.city' into a list
address_city = [dic['address.city'] for dic in list_of_flat_dic if 'address.city' in dic]

In [100]:
#percentage of the records containing the field 'address.city'
len(address_city)/len(list_of_flat_dic)

0.40822

In [107]:
from collections import Counter
import operator

#top five most common values
sorted(Counter(address_city).items(), key=operator.itemgetter(1),reverse=True)[0:5]

[('New Michael', 61),
 ('Lake Michael', 58),
 ('East Michael', 57),
 ('North Michael', 56),
 ('Port Michael', 56)]

In [103]:
#extract values of field 'address.state' into a list
address_state = [dic['address.state'] for dic in list_of_flat_dic if 'address.state' in dic]

In [104]:
#percentage of the records containing the field 'address.state'
len(address_state)/len(list_of_flat_dic)

0.40822

In [108]:
from collections import Counter
import operator

#top five most common values
sorted(Counter(address_state).items(), key=operator.itemgetter(1),reverse=True)[0:5]

[('NC', 1109), ('DC', 1106), ('MI', 1097), ('MD', 1095), ('OH', 1085)]

In [109]:
#extract values of field 'address.street' into a list
address_street = [dic['address.street'] for dic in list_of_flat_dic if 'address.street' in dic]

In [110]:
#percentage of the records containing the field 'address.street'
len(address_street)/len(list_of_flat_dic)

0.40822

In [111]:
from collections import Counter
import operator

#top five most common values
sorted(Counter(address_street).items(), key=operator.itemgetter(1),reverse=True)[0:5]

[('45662 Joshua Corner', 1),
 ('74087 Vazquez Hills', 1),
 ('503 Morton Cape', 1),
 ('418 Collins Trail Apt. 160', 1),
 ('38733 Smith Street', 1)]

In [112]:
#extract values of field 'address.zip' into a list
address_zip = [dic['address.zip'] for dic in list_of_flat_dic if 'address.zip' in dic]

In [113]:
#percentage of the records containing the field 'address.zip'
len(address_zip)/len(list_of_flat_dic)

0.40822

In [114]:
from collections import Counter
import operator

#top five most common values 
sorted(Counter(address_zip).items(), key=operator.itemgetter(1),reverse=True)[0:5]

[('98018', 5), ('99528', 5), ('17086', 5), ('26231', 5), ('50032', 5)]

In [115]:
#extract values of field 'email' into a list
email = [dic['email'] for dic in list_of_flat_dic if 'email' in dic]

In [117]:
#percentage of the records containing the field 'email'
len(email)/len(list_of_flat_dic)

0.8725333333333334

In [118]:
from collections import Counter
import operator

#top five most common values 
sorted(Counter(email).items(), key=operator.itemgetter(1),reverse=True)[0:5]

[('kwilliams@yahoo.com', 9),
 ('asmith@hotmail.com', 9),
 ('zsmith@gmail.com', 9),
 ('qsmith@gmail.com', 9),
 ('gsmith@gmail.com', 8)]

In [119]:
#extract values of field 'name' into a list
name = [dic['name'] for dic in list_of_flat_dic if 'name' in dic]

In [120]:
#percentage of the records containing the field 'name'
len(name)/len(list_of_flat_dic)

0.2870533333333333

In [121]:
from collections import Counter
import operator

#top five most common values 
sorted(Counter(name).items(), key=operator.itemgetter(1),reverse=True)[0:5]

[('David Smith', 20),
 ('John Smith', 19),
 ('Michael Smith', 18),
 ('Michael Johnson', 17),
 ('Christopher Smith', 16)]

In [124]:
#extract values of field 'name.firstname' into a list
name_firstname = [dic['name.firstname'] for dic in list_of_flat_dic if 'name.firstname' in dic]

In [125]:
#percentage of the records containing the field 'name.first'
len(name_firstname)/len(list_of_flat_dic)

0.7000266666666667

In [126]:
from collections import Counter
import operator

#top five most common values 
sorted(Counter(name_firstname).items(), key=operator.itemgetter(1),reverse=True)[0:5]

[('Michael', 2356),
 ('David', 1577),
 ('James', 1551),
 ('Jennifer', 1543),
 ('John', 1479)]

In [127]:
#extract values of field 'name.lastname' into a list
name_lastname = [dic['name.lastname'] for dic in list_of_flat_dic if 'name.lastname' in dic]

In [128]:
#percentage of the records containing the field 'name.lastname'
len(name_lastname)/len(list_of_flat_dic)

0.7000266666666667

In [129]:
from collections import Counter
import operator

#top five most common values 
sorted(Counter(name_lastname).items(), key=operator.itemgetter(1),reverse=True)[0:5]

[('Smith', 2373),
 ('Johnson', 1784),
 ('Williams', 1417),
 ('Brown', 1350),
 ('Jones', 1296)]

In [130]:
#extract values of field 'phone' into a list
phone = [dic['phone'] for dic in list_of_flat_dic if 'phone' in dic]

In [131]:
#percentage of the records containing the field 'phone'
len(phone)/len(list_of_flat_dic)

0.9350733333333333

In [132]:
from collections import Counter
import operator

#top five most common values 
sorted(Counter(phone).items(), key=operator.itemgetter(1),reverse=True)[0:5]

[('1-298-341-1025', 2),
 ('075.987.3303', 1),
 ('(336)954-0572', 1),
 ('834.799.9072x79660', 1),
 ('588-386-6769x941', 1)]

## 3. How many distinct first names appear in this data set? 

Explain your procedure for identifying distinct first names.

#Extract both 'name.firstname' and the firstname portion of field 'name' and concatenate into one list of first names
#Extract the first name portion in field 'name' into a list firstname1
#Conbine two list of firstname into one master list
#Convert list to pandas dataframe column and count unique value

In [135]:
#Extract both 'name.firstname' and the first name portion in field 'name' into one list of first names:
#Extract the first name portion in field 'name' into a list firstname1
firstname1=[]

for item in name:
    firstname1.append(item.split()[0])    

firstname1

['Morgan',
 'Bernard',
 'Casey',
 'Barbara',
 'Crystal',
 'Eric',
 'Cindy',
 'Thomas',
 'Gregory',
 'Billy',
 'David',
 'Terry',
 'Deborah',
 'Lori',
 'Daryl',
 'Kevin',
 'Katherine',
 'Alisha',
 'Blake',
 'Robert',
 'Paul',
 'Christine',
 'Christopher',
 'Jeremy',
 'Rhonda',
 'Ronald',
 'Teresa',
 'Jeremy',
 'Andrea',
 'Kenneth',
 'Lauren',
 'Marc',
 'Brett',
 'Mr.',
 'Nicole',
 'Annette',
 'Henry',
 'Heather',
 'Dean',
 'Brenda',
 'Cody',
 'Ryan',
 'Holly',
 'Fernando',
 'Jason',
 'Christine',
 'Nathan',
 'David',
 'Rickey',
 'Kimberly',
 'James',
 'Austin',
 'Alexandra',
 'Jeanette',
 'Dennis',
 'Ashley',
 'Andrew',
 'Dr.',
 'Tara',
 'Donna',
 'Nathan',
 'Diana',
 'Amber',
 'Barry',
 'Mark',
 'Jared',
 'Karen',
 'Carrie',
 'Allison',
 'Paul',
 'Thomas',
 'Kyle',
 'Christopher',
 'Michael',
 'Jeanne',
 'Michael',
 'Matthew',
 'Michael',
 'Randy',
 'Lisa',
 'Ryan',
 'Samantha',
 'William',
 'Angela',
 'Brendan',
 'Robert',
 'Donna',
 'Sheena',
 'Stephanie',
 'Christopher',
 'Ryan',
 '

In [136]:
#conbine two list of firstname into one master list:
firstname=name_firstname+firstname1
len(firstname) #148062

148062

In [144]:
#convert list to pandas dataframe column and count unique value
pd.DataFrame({'firstname': firstname})['firstname'].nunique()

695

## 4. How many distinct street names appear in this data set? 

Explain your procedure for identifying distinct street names.

1. split each element in list address_street by space into list of tokens
2. strip all numbers from tokens
3. remove_words=['Apt.', 'Apt', 'Suite', 'Unit'] from tokens
4. rejoin tokens into string by space, and append strings to be list of strings
5. convert list of strings to pandas dataframe column and count unique values

In [150]:
#1. split each string element in list by space into tokens, remove tokens consisting of numbers or remove_words
#2. concatenate back to a list of string elements

address_street_clean=[]

remove_words=['Apt.', 'Apt', 'Suite', 'Unit']
for item in address_street:
    temp_tokens=[x for x in item.split() if not (x.isdigit() or x in remove_words)]
    address_street_clean.append(' '.join(temp_tokens))

address_street_clean

['David Pass',
 'Young Square',
 'Smith Vista',
 'Knapp Unions',
 'Mills Estates',
 'Hailey Drives',
 'Baker Branch',
 'Bennett Motorway',
 'Matthew Ports',
 'Timothy Viaduct',
 'Dominguez Islands',
 'Matthew Courts',
 'Lloyd Canyon',
 'Garcia Rapids',
 'Paul Plaza',
 'Barnes Loaf',
 'Mccarthy Hill',
 'Justin Spur',
 'Miller Street',
 'Osborne Village',
 'Campbell Groves',
 'Mullins Pines',
 'Graham Meadows',
 'Solis Loop',
 'Jason Lane',
 'Bender Common',
 'Ward Lake',
 'Kenneth Trafficway',
 'Elizabeth View',
 'Kimberly Plains',
 'Cook Parks',
 'David Extension',
 'Stephanie Avenue',
 'Cynthia Cliffs',
 'Angela Turnpike',
 'Vargas Common',
 'Barrett Camp',
 'Jordan Mall',
 'Phillips Tunnel',
 'Fox Trace',
 'Mcclure Crossroad',
 'Sarah Cliff',
 'Tammy Square',
 'Mcbride Plaza',
 'Brooks Tunnel',
 'Brown Mountains',
 'Jeff Islands',
 'Harris Meadow',
 'Campbell Gardens',
 'Potts Road',
 'Jessica Mount',
 'Escobar Pike',
 'Gibson Ridge',
 'Anthony Garden',
 'Cobb Viaduct',
 'Romero Lodg

In [152]:
#count number of distinct street names in list of clean street names:
pd.DataFrame({'address_street_clean': address_street_clean})['address_street_clean'].nunique()

46349

## 5. What are the 5 most common US area codes in the phone number field? 
Explain your approach to identify the US area codes in this data set.

1. filter and find the us phone numbers from phone number field that matches forms below:

800-555-1212
800.555.1212
(800) 555-1212
1-800-555-1212
1-800-555-1212x1234
(800)555-1212x1234
800-555-1212x1234

2. extract the area code in the phone number field into a list of strings
3. convert into pandas dataframe column and count values

In [182]:
import re

phone_pattern_list=['^(\d{3})-(\d{3})-(\d{4})$', '^(\d{3}).(\d{3}).(\d{4})$','^\((\d{3})\)(\d{3})-(\d{4})$', 
               '^1-(\d{3})-(\d{3})-(\d{4})$','^1-(\d{3})-(\d{3})-(\d{4})x(\d+)$',
              '^\((\d{3})\)(\d{3})-(\d{4})x(\d+)$','^(\d{3})-(\d{3})-(\d{4})x(\d+)$']
#http://www.diveintopython.net/regular_expressions/phone_numbers.html

#extract area codes in phone number field
us_phone=[]
for pattern in phone_pattern_list:
    for numbers in phone:
        if re.match(pattern, numbers):
            us_phone.append(numbers)
us_phone    

['624-869-4610',
 '233-423-3823',
 '721-135-6630',
 '196-454-2484',
 '532-944-5053',
 '217-008-6889',
 '166-908-2192',
 '597-698-8590',
 '032-483-8114',
 '481-735-7881',
 '317-278-1828',
 '111-814-4015',
 '483-318-4001',
 '882-694-7387',
 '941-549-4192',
 '516-636-2417',
 '230-436-3623',
 '025-273-9681',
 '404-602-8435',
 '164-365-5069',
 '301-884-8878',
 '724-963-9736',
 '749-530-0941',
 '233-154-4637',
 '271-349-7484',
 '882-468-4501',
 '311-188-1288',
 '426-437-0796',
 '435-394-8102',
 '789-699-1421',
 '814-011-8526',
 '979-207-6315',
 '319-052-1349',
 '644-793-0705',
 '612-009-8377',
 '125-324-9500',
 '788-502-3042',
 '222-102-6466',
 '828-129-1316',
 '538-768-3624',
 '051-264-9904',
 '598-712-3857',
 '135-569-6738',
 '550-735-9871',
 '732-036-7489',
 '331-209-8500',
 '209-677-4888',
 '936-226-6396',
 '821-386-5905',
 '027-857-6360',
 '125-402-6265',
 '612-681-0804',
 '552-128-0921',
 '654-992-6086',
 '209-545-3016',
 '759-667-9572',
 '985-115-7766',
 '757-308-2962',
 '025-348-1283

In [183]:
len(us_phone) #111058 US phone numbers matched and extracted


111058

In [184]:
#extract area code from US phone numbers list:
#match pattern1 phone numbers of which the first three numbers are area code
pattern1=['^(\d{3})-(\d{3})-(\d{4})$', '^(\d{3}).(\d{3}).(\d{4})$','^\((\d{3})\)(\d{3})-(\d{4})$', 
              '^\((\d{3})\)(\d{3})-(\d{4})x(\d+)$','^(\d{3})-(\d{3})-(\d{4})x(\d+)$']
#match pattern2 phone numbers of which the three numbers after the first '-' are area code:
pattern2=['^1-(\d{3})-(\d{3})-(\d{4})$','^1-(\d{3})-(\d{3})-(\d{4})x(\d+)$']

area_codes1=[]
area_codes2=[]

for pattern in pattern1:
    for numbers in us_phone:
        if re.match(pattern, numbers):
            area_codes1.append(re.search('(\d+)',numbers).group(1)[0:3]) 
area_codes1

for pattern in pattern2:
    for numbers in us_phone:
        if re.match(pattern, numbers):
            area_codes2.append(numbers.split('-')[1]) 
area_codes2

['594',
 '684',
 '812',
 '288',
 '301',
 '147',
 '031',
 '147',
 '348',
 '793',
 '420',
 '209',
 '685',
 '834',
 '111',
 '493',
 '456',
 '018',
 '681',
 '443',
 '457',
 '139',
 '918',
 '338',
 '843',
 '041',
 '174',
 '905',
 '701',
 '809',
 '288',
 '541',
 '774',
 '384',
 '634',
 '344',
 '796',
 '324',
 '779',
 '070',
 '514',
 '299',
 '740',
 '695',
 '317',
 '174',
 '750',
 '260',
 '265',
 '746',
 '811',
 '311',
 '419',
 '427',
 '658',
 '251',
 '447',
 '273',
 '819',
 '662',
 '622',
 '547',
 '306',
 '934',
 '552',
 '969',
 '184',
 '149',
 '156',
 '182',
 '541',
 '963',
 '161',
 '050',
 '773',
 '418',
 '420',
 '321',
 '129',
 '746',
 '977',
 '673',
 '537',
 '976',
 '247',
 '350',
 '458',
 '798',
 '890',
 '592',
 '360',
 '942',
 '556',
 '906',
 '833',
 '123',
 '567',
 '401',
 '781',
 '373',
 '635',
 '412',
 '569',
 '895',
 '031',
 '309',
 '096',
 '899',
 '285',
 '270',
 '177',
 '922',
 '204',
 '053',
 '933',
 '297',
 '122',
 '870',
 '346',
 '431',
 '713',
 '298',
 '050',
 '019',
 '365',


In [188]:
#concatenate two sub-lists of area codes into one master list
area_codes=area_codes1+area_codes2
#convert master list of area codes into pandas dataframe column,count values and show top 5 in terms of frequency:
pd.DataFrame({'area_codes':area_codes})['area_codes'].value_counts().head(5)

092    184
947    183
390    182
712    182
593    181
Name: area_codes, dtype: int64